In [27]:
# 必要なモジュールをインポート
import os
import json
import pandas as pd
import requests
from dotenv import load_dotenv
from openai import OpenAI
from bs4 import BeautifulSoup

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [48]:
# Excelファイルを読み込む
df = pd.read_excel('../12_excel/sample.xlsx')

# 各脆弱性IDの発生頻度をカウントする
high_vulnerabilities = df[df['レベル'] == 'high']
vulnerability_counts = high_vulnerabilities['検出された脆弱性'].value_counts()

# 上位3つを抽出
top3_vulnerabilities = vulnerability_counts.head(3)

body_url = ""
for vuln_id, count in top3_vulnerabilities.items():
    url = f"https://security.snyk.io/vuln/{vuln_id}"
    
    response = requests.get(url)
    response.encoding = response.apparent_encoding

    soup = BeautifulSoup(response.text, "html.parser")
    body_html = str(soup.body)

    prompt = f"""
    以下のHTMLから脆弱性の原因パッケージ名を抽出し「検知日、脆弱性名、原因パッケージ」の形式で一覧を出力してください。一覧以外は出力しないでください。
    # 出力様式：
    Date: 検知日
    Title: 脆弱性名
    Reason: 原因パッケージ
    --------------------
    #HTML:
    {body_html}
    """

    # APIへリクエスト
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": prompt},
        ],
        max_tokens=5000,
        temperature=1
    )

    # LLMからの回答を表示
    print(response.choices[0].message.content.strip())


```
Date: 2025-04-30
Title: Improper Cleanup on Thrown Exception
Reason: org.apache.tomcat.embed:tomcat-embed-core
```
```
Date: 2025-04-24
Title: Improper Certificate Validation
Reason: org.apache.httpcomponents.client5:httpclient5
```
```
Date: 2025-03-09
Title: Path Equivalence
Reason: org.apache.tomcat.embed:tomcat-embed-core
```
